In [483]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import sklearn
import ast
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OneHotEncoder

In [484]:
df = pd.read_csv(r'./data/raw_data.csv')
df = df[df['room_type'] == 'Entire home/apt']
df.drop(['name','bathrooms','listing_url','scrape_id','last_scraped','source','description','neighborhood_overview',
         'picture_url','host_id','host_url','host_name','host_since','host_about','host_acceptance_rate',
         'host_thumbnail_url', 'host_picture_url','host_listings_count', 'host_total_listings_count',
         'host_verifications','host_has_profile_pic','host_response_time','host_location','host_neighbourhood',
         'host_identity_verified','property_type','calculated_host_listings_count_private_rooms',
         'calculated_host_listings_count_shared_rooms','reviews_per_month','license','instant_bookable',
         'calculated_host_listings_count','calculated_host_listings_count_entire_homes',
         'number_of_reviews_l30d', 'first_review', 'last_review', 'review_scores_rating',
         'review_scores_accuracy','review_scores_cleanliness','review_scores_checkin',
         'review_scores_communication','review_scores_location','calendar_updated', 'has_availability',	
         'availability_30', 'availability_60',	'availability_90', 'availability_365', 'calendar_last_scraped',
         'number_of_reviews', 'number_of_reviews_ltm','minimum_nights',	'maximum_nights',	
         'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights',	
         'maximum_maximum_nights', 'minimum_nights_avg_ntm','maximum_nights_avg_ntm','neighbourhood','room_type'],axis=1,inplace=True)
df.rename(columns={'neighbourhood_group_cleansed': 'distrito',
                   'neighbourhood_cleansed': 'barrio',
                   'neighbourhood': 'localidad',
                   'host_is_superhost':'superhost',
                   'host_response_rate':'ratio_de_respuesta',
                   'price': 'precio_septiembre'
                   }, inplace=True)
df2=pd.read_csv(r'./data/listings10JUN.csv')
df2 = df2[df2['room_type'] == 'Entire home/apt']
df['precio_junio']=df2['price']

df3=pd.read_csv(r'./data/listings10JUN.csv')
df3 = df3[df3['room_type'] == 'Entire home/apt']
df['precio_diciembre']=df3['price']

df4=pd.read_csv(r'./data/listings10JUN.csv')
df4 = df4[df4['room_type'] == 'Entire home/apt']
df['precio_marzo']=df4['price']


In [485]:
map_dict = df.set_index('id')['precio_septiembre'].to_dict()
df['precio_junio'].fillna(df['id'].map(map_dict), inplace=True)
df['precio_diciembre'].fillna(df['id'].map(map_dict), inplace=True)
df['precio_marzo'].fillna(df['id'].map(map_dict), inplace=True)
df['bedrooms'].fillna(2, inplace=True)
df['beds'].fillna(df['bedrooms'], inplace=True)
df['ratio_de_respuesta'].fillna(0, inplace=True)
df['review_scores_value'].fillna(4.77, inplace=True)
df['bathrooms_text'].dropna()

2           1 bath
3           1 bath
4           1 bath
6           1 bath
7           1 bath
           ...    
24822       1 bath
24824       1 bath
24825    1.5 baths
24826       1 bath
24827      2 baths
Name: bathrooms_text, Length: 15638, dtype: object

In [486]:
df['superhost'] = df['superhost'].replace({'t': 1, 'f': 0})

In [487]:
df['precio_junio'] = df['precio_junio'].str.replace('$', '')
df['precio_septiembre'] = df['precio_septiembre'].str.replace('$', '')
df['precio_diciembre'] = df['precio_diciembre'].str.replace('$', '')
df['precio_marzo'] = df['precio_marzo'].str.replace('$', '')
df['precio_junio'] = df['precio_junio'].str.replace(',', '')
df['precio_septiembre'] = df['precio_septiembre'].str.replace(',', '')
df['precio_diciembre'] = df['precio_diciembre'].str.replace(',', '')
df['precio_marzo'] = df['precio_marzo'].str.replace(',', '')
df['ratio_de_respuesta'] = df['ratio_de_respuesta'].str.replace('%', '')
df['bathrooms_text']=df['bathrooms_text'].str.replace(' bath','')
df['bathrooms_text']=df['bathrooms_text'].str.replace('s','')
df['bathrooms_text']=df['bathrooms_text'].str.replace('Half-bath','0.5')
df['room_type']=1
df['localidad']='Madrid, España'

In [488]:
df['ratio_de_respuesta'].fillna(4.77, inplace=True)

In [489]:
df['precio_junio'] = df['precio_junio'].astype(float)
df['precio_septiembre'] = df['precio_septiembre'].astype(float)
df['precio_diciembre'] = df['precio_diciembre'].astype(float)
df['precio_marzo'] = df['precio_marzo'].astype(float)
df['bedrooms']=df['bedrooms'].astype(int)
df['beds']=df['beds'].astype(int)
df['ratio_de_respuesta']=df['ratio_de_respuesta'].astype(int)
df['bathrooms_text']=df['bathrooms_text'].astype(float)
df['superhost']=df['superhost'].astype(bool)
df['room_type']=df['room_type'].astype(int)

In [490]:
df['precio_total'] = df[['precio_septiembre', 'precio_junio', 'precio_diciembre', 'precio_marzo']].median(axis=1)

In [491]:
df.drop(['localidad','barrio','precio_septiembre','precio_junio','precio_diciembre','precio_marzo','latitude','longitude'],axis=1,inplace=True)

In [492]:
df['bathrooms_text'] = df['bathrooms_text'].dropna()


In [493]:
label_encoder = LabelEncoder()

# Ajustar y transformar la columna 'superhost'
df['distrito'] = label_encoder.fit_transform(df['distrito'])


In [494]:
df['amenities'] = df['amenities'].astype(str)

# Usa el método str.contains para verificar si 'air conditioning' está presente en cada fila de la columna 'amenities'
# y luego usa sum() para contar el número total de True (True cuenta como 1, False como 0)
num_air_conditioning = df['amenities'].str.contains('Air conditioning', case=False).sum()
num_air_conditioning

10788

In [495]:
df['amenities'] = df['amenities'].apply(lambda x: ast.literal_eval(x))

In [496]:
df['amenities'] = df['amenities'].apply(lambda x: [tag.strip() for tag in x])

In [497]:
suma = 0
lista = ['Kitchen', 'Elevator', 'Wifi', 'City skyline view', 'Air conditioning']

for row_index, amenities_str in enumerate(df['amenities']):
    if row_index < len(df):
        # Inicializa un diccionario para almacenar el estado de las amenities en la fila actual
        row_amenities = {amenity: False for amenity in lista}
        
        # Itera sobre la lista de amenities y marca como True si está presente en la cadena
        for amenity in lista:
            if amenity in amenities_str:
                row_amenities[amenity] = True

        # Asigna los valores al DataFrame para la fila actual
        for amenity, value in row_amenities.items():
            df.at[row_index, amenity] = value

        suma += sum(row_amenities.values())  # Incrementa suma por la cantidad de True en la fila
    else:
        break



In [498]:
df= df.drop(['amenities','id'],axis=1)

In [499]:
df= df.dropna(axis=0)

In [500]:
df.to_csv('./data/treated_data.csv', index=False)